In [26]:
import os
import openai
import numpy as np
from IPython.display import display, HTML

In [27]:
api_key = os.getenv("AAPELI_KEY")

In [28]:
client = openai.Client(api_key=api_key, base_url="https://px.axess-ai.com/v1")

In [49]:
client.models.list().data

[Model(id='anthropic.claude-3.5-haiku', created=1677610602, object='model', owned_by='openai'),
 Model(id='qwen.qwq-32b', created=1677610602, object='model', owned_by='openai'),
 Model(id='qwen.qwen3-235b-a22b', created=1677610602, object='model', owned_by='openai'),
 Model(id='mistral.mistral-large-2411', created=1677610602, object='model', owned_by='openai'),
 Model(id='google.gemma-3-27b', created=1677610602, object='model', owned_by='openai'),
 Model(id='xai.grok-3-beta-mini', created=1677610602, object='model', owned_by='openai'),
 Model(id='meta.llama-3.3-70b', created=1677610602, object='model', owned_by='openai'),
 Model(id='xai.grok-3-beta', created=1677610602, object='model', owned_by='openai'),
 Model(id='google.gemini-2.5-pro-preview', created=1677610602, object='model', owned_by='openai'),
 Model(id='meta.llama-4-scout', created=1677610602, object='model', owned_by='openai'),
 Model(id='openai.gpt-4.1-nano', created=1677610602, object='model', owned_by='openai'),
 Model(id

In [50]:
def get_completion(
    messages: list[dict[str, str]],
    model: str = "meta.llama-3.3-70b",
    max_tokens=500,
    temperature=0,
    stop=None,
    seed=123,
    tools=None,
    logprobs=None,  # whether to return log probabilities of the output tokens or not. If true, returns the log probabilities of each output token returned in the content of message..
    top_logprobs=None,
) -> str:
    params = {
        "model": model,
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "stop": stop,
        "seed": seed,
        "logprobs": logprobs,
        "top_logprobs": top_logprobs,
    }
    if tools:
        params["tools"] = tools

    completion = client.chat.completions.create(**params)
    return completion

In [56]:
CLASSIFICATION_PROMPT = """You will be given a headline of a news article.
Classify the article into one of the following categories: Technology, Politics, Sports, and Art.
Return only the name of the category, and nothing else.
MAKE SURE your output is one of the four categories stated.
Article headline: {headline}"""

In [52]:
CLASSIFICATION_PROMPT = """You will be given a headline of a news article.
Classify the article into one of the following categories: Technology, Politics, Sports, and Art.
If the category is Technology, return A. If it is Politics, return B. If it is Sports, return C. If it is Art, return D.
MAKE SURE your output is one of the four letters, A, B, C, or D.
Article headline: {headline}""" 

In [53]:
CLASSIFICATION_PROMPT = """You will be given a headline of a news article.
Classify the article into one of the following categories: Technology, Politics, Sports, and Art.
Article headline: {headline}""" 

In [57]:
headlines = [
    "Tech Giant Unveils Latest Smartphone Model with Advanced Photo-Editing Features.",
    "Local Mayor Launches Initiative to Enhance Urban Public Transport.",
    "Tennis Champion Showcases Hidden Talents in Symphony Orchestra Debut",
]

In [58]:
for headline in headlines:
    print(f"\nHeadline: {headline}")
    API_RESPONSE = get_completion(
        [{"role": "user", "content": CLASSIFICATION_PROMPT.format(headline=headline)}],
        model="openai.gpt-4o-mini",
    )
    print(f"Category: {API_RESPONSE.choices[0].message.content}\n")


Headline: Tech Giant Unveils Latest Smartphone Model with Advanced Photo-Editing Features.
Category: Technology


Headline: Local Mayor Launches Initiative to Enhance Urban Public Transport.
Category: Politics


Headline: Tennis Champion Showcases Hidden Talents in Symphony Orchestra Debut
Category: Art



In [62]:
for headline in headlines:
    print(f"\nHeadline: {headline}")
    API_RESPONSE = get_completion(
        [{"role": "user", "content": CLASSIFICATION_PROMPT.format(headline=headline)}],
        model="meta.llama-3.3-70b",
        logprobs=True,
        top_logprobs=21,
        allowed_openai_params=['logprobs', 'top_logprobs']
    )
    top_two_logprobs = API_RESPONSE.choices[0].logprobs.content[0].top_logprobs
    html_content = ""
    for i, logprob in enumerate(top_two_logprobs, start=1):
        html_content += (
            f"<span style='color: cyan'>Output token {i}:</span> {logprob.token}, "
            f"<span style='color: darkorange'>logprobs:</span> {logprob.logprob}, "
            f"<span style='color: magenta'>linear probability:</span> {np.round(np.exp(logprob.logprob)*100,2)}%<br>"
        )
    display(HTML(html_content))
    print("\n")


Headline: Tech Giant Unveils Latest Smartphone Model with Advanced Photo-Editing Features.


TypeError: get_completion() got an unexpected keyword argument 'allowed_openai_params'